In [3]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [4]:
df = pd.read_csv("../../data/dunnhumby_BatF_transactions.csv")

In [9]:

# Einfache Bereinigung und Datentyp-Konvertierung
df['WEEK_END_DATE'] = pd.to_datetime(df['WEEK_END_DATE'])

# Berechnung der Quantile für 'SPEND' innerhalb jeder 'STORE_NUM'
df['Umsatz_Quantil'] = df.groupby('STORE_NUM')['SPEND'].transform(
    lambda x: pd.qcut(x, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
)

# Einen Dash-App-Server einrichten
app = dash.Dash(__name__)

# Dash-Layout
app.layout = html.Div([
    html.H1("Produktumsatz-Dashboard"),
    dcc.Dropdown(
        id='product-dropdown',
        options=[{'label': i, 'value': i} for i in df['UPC'].unique()],
        value=df['UPC'].unique()[0]  # Erstes Produkt als Standardwert
    ),
    dcc.Graph(id='revenue-time-series-graph'),
    dcc.Dropdown(
        id='quantile-dropdown',
        options=[{'label': q, 'value': q} for q in ['Q1', 'Q2', 'Q3', 'Q4']],
        value='Q1'  # Erstes Quantil als Standardwert
    ),
    dcc.Graph(id='marketing-impact-graph'),
])

# Callbacks für interaktive Inhalte
@app.callback(
    Output('revenue-time-series-graph', 'figure'),
    [Input('product-dropdown', 'value')]
)
def update_time_series(selected_product):
    # Daten für das ausgewählte Produkt filtern
    filtered_df = df[df['UPC'] == selected_product]
    
    # Umsatz-Zeitreihendiagramm erstellen
    fig = px.line(filtered_df, x='WEEK_END_DATE', y='SPEND', title='Umsatz über die Zeit')
    return fig

@app.callback(
    Output('marketing-impact-graph', 'figure'),
    [Input('product-dropdown', 'value'),
     Input('quantile-dropdown', 'value')]
)
def update_marketing_impact(selected_product, selected_quantile):
    # Daten für das ausgewählte Produkt und Quantil filtern
    filtered_df = df[(df['UPC'] == selected_product) & (df['Umsatz_Quantil'] == selected_quantile)]
    
    # Berechnen des medianen Umsatzes ohne und mit Werbemaßnahmen
    median_spend_no_ad = filtered_df[filtered_df['FEATURE'] == 0]['SPEND'].median()
    median_spend_with_ad = filtered_df[filtered_df['FEATURE'] == 1]['SPEND'].median()
    
    # Bar-Graphen für den Vergleich der medianen Umsätze erstellen
    fig = px.bar(
        x=['Ohne Werbung', 'Mit Werbung'],
        y=[median_spend_no_ad, median_spend_with_ad],
        title='Impact der Werbemaßnahme auf den Umsatz'
    )
    return fig

In [10]:
app.run(jupyter_mode="tab", debug=True, host='127.0.0.1', port='8081')

Dash app running on http://127.0.0.1:8081/


<IPython.core.display.Javascript object>